Autor: Jakub Białek

Data: 2020-14-05

In [72]:
from IPython.display import HTML

#### Przygotowanie modeli i krzywych CP.

Zmienna objaśniana (wydatki na opiekę zdrowotną) została przekształcona z wykorzystaniem logarytmu o podstawie 3 (blisko logarytmu naturalnego, ale łatwiejsze w interpretacji). W związku z tym interpretacja jest następująca - jeśli zmiana jakiegoś wejścia spowodowała wzrost predykcji o $n$ tzn. że wydatki na OZ wzrosły o $3^n$.

In [73]:
import pandas as pd
import numpy as np
import os

import shap
import xgboost as xgb
from sklearn.linear_model import Ridge
from sklearn.linear_model import LinearRegression

from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.metrics import precision_score, recall_score, accuracy_score, roc_curve, \
roc_auc_score, precision_recall_curve, confusion_matrix, r2_score, mean_absolute_error, \
mean_squared_error
import pandas_profiling as pp
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:60% !important; }</style>"))

In [74]:
pd.options.display.max_columns = 200

In [75]:
!dir data

 Volume in drive C is Windows
 Volume Serial Number is 5441-C8EF

 Directory of C:\Users\bialekj\Documents\Moje\Doktorat wdrozeniowy\wyklady\XAI\pd4\data

15.04.2020  14:46    <DIR>          .
15.04.2020  14:46    <DIR>          ..
22.03.2020  18:25         2˙452˙351 MEPS_data_preprocessed.csv
               1 File(s)      2˙452˙351 bytes
               2 Dir(s)  43˙349˙794˙816 bytes free


In [76]:
data_dir = 'data'
df = pd.read_csv(os.path.join(data_dir, 'MEPS_data_preprocessed.csv'))
df.reset_index(drop=True, inplace=True)
df_raw = df.copy()

In [77]:
df = df.drop('PANEL', axis=1)

In [78]:
y = df.pop('HEALTHEXP')

In [79]:
y_loge = np.log(y)
y_loge[y==0] = 0
y_log3 = y_loge / np.log(3) 
y_log3 = np.asarray(y_log3)
y = np.asarray(y)

c:\users\bialekj\appdata\local\programs\python\python36\lib\site-packages\pandas\core\series.py:679: RuntimeWarning:

divide by zero encountered in log



In [80]:
categorical_features = ['REGION','MARRY31X','EDRECODE','FTSTU31X','ACTDTY31','HONRDC31',
            'RTHLTH31','MNHLTH31','HIBPDX','CHDDX','ANGIDX','MIDX','OHRTDX','STRKDX',
            'EMPHDX','CHBRON31','CHOLDX','CANCERDX','DIABDX','JTPAIN31','ARTHDX',
            'ARTHTYPE','ASTHDX','ADHDADDX','PREGNT31','WLKLIM31','ACTLIM31','SOCLIM31',
            'COGLIM31','DFHEAR42','DFSEE42','ADSMOK42','PHQ242','EMPST31','POVCAT15','INSCOV15','RACE3','GENDER']

numerical_features= [feat for feat in df.columns if feat not in categorical_features]

In [81]:
numerical_transformer = Pipeline(
    steps=[
        ('scaler', StandardScaler())
    ]
)

categorical_transformer = Pipeline(
    steps=[
        ('onehot', OneHotEncoder(handle_unknown='ignore'))
    ]
)

preprocessor = ColumnTransformer(
    transformers=[
        ('num', numerical_transformer, numerical_features),
        ('cat', categorical_transformer, categorical_features)
    ]
)

regressor = GradientBoostingRegressor(n_estimators=300, random_state=42)

mdl = Pipeline(steps=[('preprocessor', preprocessor),
                      ('regressor', regressor)])

In [82]:
X_train, X_test, y_train, y_test = train_test_split(df, y_log3, test_size=0.2, random_state=42)
mdl.fit(X_train,y_train)

Pipeline(memory=None,
         steps=[('preprocessor',
                 ColumnTransformer(n_jobs=None, remainder='drop',
                                   sparse_threshold=0.3,
                                   transformer_weights=None,
                                   transformers=[('num',
                                                  Pipeline(memory=None,
                                                           steps=[('scaler',
                                                                   StandardScaler(copy=True,
                                                                                  with_mean=True,
                                                                                  with_std=True))],
                                                           verbose=False),
                                                  ['AGE31X', 'PCS42', 'MCS42',
                                                   'K6SUM42', 'INCOME_M',
                                    

In [83]:
y_pred = mdl.predict(X_test)
r2_score(y_test,y_pred)

0.3756526032677451

In [84]:
model1 = mdl

In [85]:
regressor = LinearRegression()
mdl = Pipeline(steps=[('preprocessor', preprocessor),
                      ('regressor', regressor)])

In [86]:
mdl.fit(X_train, y_train)

Pipeline(memory=None,
         steps=[('preprocessor',
                 ColumnTransformer(n_jobs=None, remainder='drop',
                                   sparse_threshold=0.3,
                                   transformer_weights=None,
                                   transformers=[('num',
                                                  Pipeline(memory=None,
                                                           steps=[('scaler',
                                                                   StandardScaler(copy=True,
                                                                                  with_mean=True,
                                                                                  with_std=True))],
                                                           verbose=False),
                                                  ['AGE31X', 'PCS42', 'MCS42',
                                                   'K6SUM42', 'INCOME_M',
                                    

In [87]:
y_pred = mdl.predict(X_test)
r2_score(y_test,y_pred)

0.3517170921229019

In [88]:
model2 = mdl

#### Model Gradient Boosting Regressor

In [89]:
import dalex as dx 

In [90]:
model1

Pipeline(memory=None,
         steps=[('preprocessor',
                 ColumnTransformer(n_jobs=None, remainder='drop',
                                   sparse_threshold=0.3,
                                   transformer_weights=None,
                                   transformers=[('num',
                                                  Pipeline(memory=None,
                                                           steps=[('scaler',
                                                                   StandardScaler(copy=True,
                                                                                  with_mean=True,
                                                                                  with_std=True))],
                                                           verbose=False),
                                                  ['AGE31X', 'PCS42', 'MCS42',
                                                   'K6SUM42', 'INCOME_M',
                                    

In [91]:
exp = dx.Explainer(model1, df, y_log3, label = "GBR")

Preparation of a new explainer is initiated

  -> label             : GBR
  -> data              : 18350 rows 44 cols
  -> target variable   : 18350 values
  -> predict function  : <function yhat_default at 0x0000024968C36598> will be used
  -> predicted values  : min = -0.5263935188373152, mean = 5.712206973487643, max = 10.336867422601017
  -> residual function : difference between y and yhat
  -> residuals         : min = -9.516194862227534, mean = 0.003236789297550328, max = 7.676258789484571
  -> model_info        : package sklearn

A new explainer has been created!


* INCOME_M - dochód pacjenta
* K6SUM42 - ogólna ocena stanu zdrowia psychicznego z ostatnich 30 dni
* MCS42 - ocena stanu zdrowia psychicznego (na podstawie wyników z ankiety)
* PCS42 - ocena stanu zdrowia fizycznego (na podstawie wyników z ankiety)
* AGE31 - wiek

In [98]:
pdp_cat = exp.model_profile(type = 'partial', variable_type='numerical',
                            variables = [ 'PCS42', 'MCS42', 'K6SUM42', 'INCOME_M',])
pdp_cat.result['_label_'] = 'pdp'

ale_cat = exp.model_profile(type = 'accumulated', variable_type='numerical', 
                            variables = [ 'PCS42', 'MCS42', 'K6SUM42', 'INCOME_M',])
ale_cat.result['_label_'] = 'ale'
fig = ale_cat.plot(pdp_cat, size = 4, facet_ncol=2)

Calculating accumulated dependency!: 100%|███████████████████████████████████████████████| 4/4 [00:01<00:00,  3.00it/s]


Dochody pacjenta wydają się mieć niewielki wpływ na wydatki na opiekę zdrowotną, choć widać niewielką tendencję wzrostową. W przypadku wskaźników oceniających stan zdrowia, najbardziej istotny wydaje się PCS42 (ocena stanu zdrowia fizycznego). Wskaźnik ten obliczany jest na podstawie ankiety z konkretnymi pytaniami, następnie na podstawie odpowiedzi obliczany jest wynik. Im większa wartość, tym lepszy stan zdrowia (i jak widać - mniejsze koszty). W podobny sposób opracowywany jest MCS42, z tym że dotyczy zdrowia psychicznego. Okazuje się, jednak, że ma on mniejszy wpływ na koszty. Jeśli zaś chodzi o K6SUM42 (ogólne samopoczucie psychiczne z ostatnich 30 dni), to wskaźnik wydaje mi się mało wiarygodny. Po pierwsze jego sposób obliczania jest dość subiektywny (6 pytań w stylue - "Jak często przez ostatnie dni byłeś nerwowy? Odpowiedzi 0 - wcale, 1 - chwilami, 2 - w miarę często, 3- większość czasu, 4 - cały czas, suma odpowiedzi daje wynik). Widać, że sam model też miał problemy z określeniem wpływu na wydatki (spodziewalibyśmy się monotonicznego wzrostu, a tymczasem mamy dość wyraźny "dołek" w okolicach 11). Być może warto rozważyć binaryzację - widać spory skok na początku, który różnicuje tych, którzy po prostu czują się dobrze, od tych, którzy mają problemy ze stresem, motywacją itd. 

Ogólna uwaga jest taka, że interpretację wpływu zmiennych utrudnia fakt, iż niektóre zmienne wraz ze wzrostem wartości wskazują na pogorszenie stanu zdrowia (K6SUM42) a inne na jego poprawę (PCS42). 

In [99]:
pdp_cat = exp.model_profile(type = 'partial', variable_type='numerical',
                            variables = ["AGE31X"])
pdp_cat.result['_label_'] = 'pdp'

ale_cat = exp.model_profile(type = 'accumulated', variable_type='numerical', 
                            variables = ["AGE31X"])
ale_cat.result['_label_'] = 'ale'
fig = ale_cat.plot(pdp_cat, size = 4, facet_ncol=1)

Calculating accumulated dependency!: 100%|███████████████████████████████████████████████| 1/1 [00:00<00:00,  3.64it/s]


#### Model liniowy

In [63]:
model2

Pipeline(memory=None,
         steps=[('preprocessor',
                 ColumnTransformer(n_jobs=None, remainder='drop',
                                   sparse_threshold=0.3,
                                   transformer_weights=None,
                                   transformers=[('num',
                                                  Pipeline(memory=None,
                                                           steps=[('scaler',
                                                                   StandardScaler(copy=True,
                                                                                  with_mean=True,
                                                                                  with_std=True))],
                                                           verbose=False),
                                                  ['AGE31X', 'PCS42', 'MCS42',
                                                   'K6SUM42', 'INCOME_M',
                                    

In [100]:
exp = dx.Explainer(model2, df, y_log3, label = "GBR")

Preparation of a new explainer is initiated

  -> label             : GBR
  -> data              : 18350 rows 44 cols
  -> target variable   : 18350 values
  -> predict function  : <function yhat_default at 0x0000024968C36598> will be used
  -> predicted values  : min = 0.1841006146568871, mean = 5.714977408746465, max = 12.242035685091384
  -> residual function : difference between y and yhat
  -> residuals         : min = -9.674257016665672, mean = 0.0004663540387282032, max = 7.242568473035798
  -> model_info        : package sklearn

A new explainer has been created!


In [103]:
pdp_cat = exp.model_profile(type = 'partial', variable_type='numerical',
                            variables = ["AGE31X"])
pdp_cat.result['_label_'] = 'pdp'

ale_cat = exp.model_profile(type = 'accumulated', variable_type='numerical', 
                            variables = ["AGE31X"])
ale_cat.result['_label_'] = 'ale'

Calculating accumulated dependency!: 100%|███████████████████████████████████████████████| 1/1 [00:00<00:00,  3.83it/s]


In [104]:
fig = ale_cat.plot(pdp_cat, size = 4, facet_ncol=1)
# fig3.write_image("06_PCS42_pdp_ale.png", scale = 2)

Po raz kolejny z premedytacją wybieram model liniowy, chcąc zweryfikować czy na pewno w wyjaśnieniu będzie linia prosta. Tak jest i tym razem. Co ciekawe, w modelu GBR widzimy duży spadek dla noworodków (tzn. wartość przy '0' jest wysoka, ale później nagle maleje), a następnie, wraz ze wzrostem wieku powoli rośnie (z kilkoma zaburzeniami i schodkami). Innymi słowy - w pierwszej części krzywa wskazuje na mocny spadek, w drugiej na powolny wzrost, który ostatecznie nie odbudowuje tego początkowego spadku (a więc spadek jest większy niż wzrost). Można by więc przypuszczać, że model liniowy odwzoruje właśnie ten spadek. Jest jednak inaczej - model liniowy wskazuje na wzrost kosztów z wiekiem. Dzieje się tak zapewne dlatego, że ten spadek obserwowalny jest jedynie dla części obserwacji, w bardzo wąskim zakresie wartości wieku (0-3, mniej niż 4 % wszystkich obserwacji), podczas gdy wzrost występuje w całym pozostałym okresie.

In [106]:
len(df[df['AGE31X']<3])/len(df)

0.03493188010899183